## Text Cleaning

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from nltk.corpus import stopwords
import string

sc = SparkContext()
spark = SparkSession.builder.appName('TextCleaning').getOrCreate()

In [2]:
stop_words=set(stopwords.words('english'))
list_punct=list(string.punctuation)
lines=sc.textFile('data.txt')

In [3]:
m = ( lines.flatMap(lambda line: line.split(' '))
     .filter(lambda punct: punct not in list_punct)
     .filter(lambda word: word.lower() not in stop_words and word != '')
     .filter(lambda word: word[0:4].lower() != 'b\'rt' and word[0:4].lower() != 'b"rt')
     .filter(lambda word: word[0] != '@' and word[0] != '\\' and word[0] != '#' and word[0] != '&')
     .filter(lambda word: word[0:8] != 'https://' and word[0:7] != 'http://')
     .filter(lambda word: len(word) > 2)
     .filter(lambda word: '\\' not in word)
     .map(lambda word: (word.lower(), 1))
     .reduceByKey(lambda x, y: x + y))
    
df = spark.createDataFrame(m, ['word', 'count'])
df.createOrReplaceTempView('words')
spark.sql( 'show tables from default' ).show()
words = spark.sql('select word, count from words order by count desc limit 20')
words.show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |    words|       true|
+--------+---------+-----------+

+-----------+-----+
|       word|count|
+-----------+-----+
|     guitar|  421|
|    playing|   68|
|       play|   57|
|    singing|   48|
|       like|   40|
|performance|   38|
|   kyungsoo|   38|
|   chanyeol|   38|
|     missed|   38|
|     tuning|   36|
|       drop|   36|
|       name|   33|
|       alex|   33|
|   pitching|   33|
|     vocals|   33|
|        dog|   33|
|  waannnnma|   33|
|       kiss|   33|
|        one|   28|
|        new|   27|
+-----------+-----+

